
### **Step 1: Контроль качества ридов с помощью FastQC**

**a. Создание conda-окружения `fastq`**  
```bash
conda create -n fastqc
```

**b. Активация окружения и установка `FastQC`**  
1. **Активация окружения**  
   ```bash
   conda activate fastqc
   ```
2. **Установка `FastQC` (FastQC v0.12.1)**  
   ```bashc
   mamba install -c conda-forge -c bioconda -c defaults fastqc
   ```



**c-d. Запуск `FastQC` на 6 файлах и сохранение отчетов**  
Относительный путь к локальной папке с файлами: `../../data_srr/`
```bash
fastqc ../../data_srr/SRR292678sub_S1_L001_R1_001.fastq ../../data_srr/SRR292678sub_S1_L001_R2_001.fastq ../../data_srr/SRR292770_S1_L001_R1_001.fastq ../../data_srr/SRR292770_S1_L001_R2_001.fastq ../../data_srr/SRR292862_S2_L001_R1_001.fastq ../../data_srr/SRR292862_S2_L001_R2_001.fastq -o ./fastqc
```

**Удаление ненужных `.zip` файлов**
```bash
rm *.zip
```
**git** 
```bash
git add .
git commit -m 'Добавление 1 этапа'
git push
```


**Описание этапа**  
Этот этап выполнялся для проверки качества ридов с помощью FastQC, чтобы убедиться в их пригодности для дальнейшего анализа. Проверка позволяет выявить проблемы, такие как низкое качество чтений или наличие адаптерных последовательностей, которые могут повлиять на последующие этапы обработки данных.

---

### **Step 2. Сборка генома**

**a. Создание conda-окружения `spades`**  
```bash
conda create -n spades
```

**b. Активация окружения и установка `SPAdes` (SPAdes genome assembler v4.0.0)**  
1. **Активация окружения**  
   ```bash
   conda activate spades
   ```
2. **Установка `SPAdes`**  
   ```bash
   mamba install -c conda-forge -c bioconda -c defaults spades
   ```

**c-d. Сборка генома используя только paired-end риды (SRR292678) и сохранение файлов contigs.fasta и scaffolds.fasta в папку hw2/spades**  
```bash
spades.py -1 ../../data_srr/SRR292678sub_S1_L001_R1_001.fastq -2 ../../data_srr/SRR292678sub_S1_L001_R2_001.fastq -o ./spades
```
так, дальше шла подчистка всех файлов и удаление ненужного 
```bash
find . -maxdepth 1 -type f ! -name 'contigs.fasta' ! -name 'scaffolds.fasta' -exec rm {} +
find . -maxdepth 1 -type d ! -name '.' -exec rm -r {} +
```

**git** 
```bash
git add .
git commit -m 'Добавление 2 этапа'
git push

git add .
git commit -m 'Добавление во 2 этап журнала lab_journal.ipynb '
git push
```
Два комита так как в первый не попал журнал с записями (отправляла git add . из папки spades)

---

### **Step 3. Оценка качества сборки**

**a. Создание conda-окружения для `quast`**  
Для оценки качества сборки создаем отдельное окружение для **QUAST** с помощью следующей команды:
```bash
conda create -n quast
```

**b. Активация окружения и установка `QUAST` (v5.2.0)**  
1. **Активация окружения**  
   После создания окружения активируем его:
   ```bash
   conda activate quast
   ```
2. **Установка `QUAST`**  
   Используем `mamba` для быстрой установки:
   ```bash
   mamba install -c conda-forge -c bioconda -c defaults quast
   ```

**c-d. Анализ файлов сборки с помощью `QUAST` и сохрание в `hw2/quast`**  
Для оценки качества сборки с использованием **QUAST** запускаем команду, анализирующую файлы `contigs.fasta` и `scaffolds.fasta`, и сохраняем отчеты в папку `hw2/quast`:
```bash
quast ./spades/contigs.fasta ./spades/scaffolds.fasta -o ./quast -t 1
```

**Результаты анализа**  
Основные метрики, полученные из отчета **QUAST**:
- **N50**: 105346
- **Общая длина сборки**: 5244705 bp 
- **Количество контингов**: 206 

**git**  

```bash
git add .
git commit -m 'Добавление 3 этапа'
git push
```

---

### **Step 4. Аннотация генома**

**a. Создание conda-окружения `prokka`**  
```bash
conda create -n prokka
```

**b. Активация окружения и установка `Prokka` (prokka 1.14.6)**  
1. **Активация окружения**  
   ```bash
   conda activate prokka
   ```
2. **Установка `Prokka`**  
   ```bash
   mamba install -c conda-forge -c bioconda -c defaults prokka
   ```
**c-d. Выполнение аннотации файла scaffolds.fasta и cохранение результатов в папку hw2/prokka**  
```bash
prokka ./spades/scaffolds.fasta -o ./prokka --force
```

**Статистика аннотации**  
- organism: Genus species strain 
- contigs: 499
- bases: 5316783
- CDS: 4970
- rRNA: 5
- repeat_region: 1
- tRNA: 69
- tmRNA: 1

**git**  

```bash
git add .
git commit -m 'Добавление 4 этапа'
git push
```

---

### **Step 5. Поиск генов устойчивости и вирулентности**

**a. Создание conda-окружения `abricate_env`**  
```bash
conda create -n abricate_env
```

**b. Активация окружения и установка `ABRicate` (abricate 1.0.1)**  
1. **Активация окружения**  
   ```bash
   conda activate abricate_env
   ```
2. **Установка `ABRicate`**  
   ```bash
   mamba install -c conda-forge -c bioconda -c defaults abricate
   ```
**c-d. Выполнение поиска по базам данных NCBI и ResFinder и сохранение результатов в папку hw2/abricate**  
```bash
abricate --db ncbi ./spades/scaffolds.fasta --csv > ./abricate/ncbi_results.csv
abricate --db resfinder ./spades/scaffolds.fasta --csv > ./abricate/resfinder_results.csv

```

**Найденные гены: Результаты по базе NCBI:**
- blaTEM-1
- blaEC-18
- blaCTX-M-15
- sul1
- dfrA7
- aph(6)-Id
- aph(3'')-Ib
- sul2
- tet(A)

**Найденные гены: Результаты по базе ResFinder:**
- blaTEM-1B_1
- mdf(A)_1
- blaCTX-M-15_1
- sul1_5
- dfrA7_5
- aph(6)-Id_1
- aph(3'')-Ib_5
- sul2_3
- tet(A)_6

**Ключевые гены устойчивости и вирулентности:**

1. **Гены устойчивости к антибиотикам**:
   - **blaTEM-1, blaTEM-1B_1**: Гены, кодирующие бета-лактамазы широкого спектра действия, обеспечивающие устойчивость к бета-лактамным антибиотикам, таким как амоксициллин и ампициллин.
   - **blaEC-18 и blaCTX-M-15, blaCTX-M-15_1**: Гены, кодирующие бета-лактамазы расширенного спектра действия, обеспечивающие устойчивость к цефалоспоринам.
   - **sul1, sul1_5 и sul2, sul2_3**: Гены, связанные с устойчивостью к сульфаниламидам (например, сульфаметоксазол).
   - **dfrA7, dfrA7_5**: Гены, кодирующие резистентные к триметоприму ферменты дигидрофолатредуктазы.
   - **aph(6)-Id, aph(6)-Id_1 и aph(3'')-Ib, aph(3'')-Ib_5**: Гены, кодирующие аминотрансферазы, обеспечивающие устойчивость к стрептомицину.
   - **tet(A), tet(A)_6**: Гены, ответственные за устойчивость к тетрациклину за счет эффлюксных транспортеров.
   - **mdf(A)_1**: Ген, связанный с множественной устойчивостью к антибиотикам.

2. **Ключевые гены устойчивости**:
   - **blaTEM-1, blaCTX-M-15** — обеспечивают устойчивость к бета-лактамным антибиотикам.
   - **sul1, sul2** — устойчивость к сульфаниламидам.
   - **tet(A)** — устойчивость к тетрациклинам.
   - **aph(6)-Id и aph(3'')-Ib** — устойчивость к аминогликозидам (стрептомицин).
   - **mdf(A)** — множественная устойчивость.

**git**  

```bash
git add .
git commit -m 'Добавление 5 этапа'
git push
```

---
